# 1. 웹크롤링

In [ ]:
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import shutil

try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제
except FileNotFoundError:
    pass

subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동


option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

## 1.1 아이템 크롤링

In [ ]:
%%time


# 제품 url들 수집

name_list = []
num_list =[]
rating_list=[]
rev_url_list=[]

try:
    for j in range(0,3):
        driver.get('https://kr.iherb.com/c/mct-oil?p={}'.format(j+1))
        for i in range(0,24):

            item = "div.product-inner.product-inner-wide > div.absolute-link-wrapper > a"
            item_raw = driver.find_elements_by_css_selector(item)

            rate=".rating-count"
            rating_raw =driver.find_elements_by_css_selector(rate)

            # 상품번호 크롤링
            num = item_raw[i].get_attribute('data-product-id')
            num_list.append(num)
            # 제목 크롤링 
            name = item_raw[i].get_attribute('title')
            name_list.append(name)

            #평점 크롤링
            rating=rating_raw[i].get_attribute('title')
            rating_list.append(rating)
            #리뷰창 크롤링
            review=rating_raw[i].get_attribute('href')
            rev_url_list.append(review)

            print(i,name,rating)
            time.sleep(2)
        
except:
    driver.close()

In [ ]:
df1 = pd.DataFrame({'name':name_list, 'item_id':num_list})
df1

In [ ]:
df2 = pd.DataFrame({"평점":rating_list,"리뷰url":rev_url_list})
df2

In [ ]:
result = pd.concat([df1,df2],axis=1)
result

### 1.1.1 데이터전처리

In [ ]:
평점 = result.평점.str.split(' -').str[0].tolist()
평점수 = result.평점.str.split(' -').str[1].tolist()
result["평점"]=평점
result["평점수"]=평점수
result

In [ ]:
평점= result.평점.str.split('/').str[0].tolist()
평점수=result.평점수.str.split(" ").str[1].tolist()

In [ ]:
result["평점"]=평점
result["평점수"]=평점수
result

In [ ]:
result["평점"][47]="4.7"
result["평점수"][47]="584"
result.isnull().sum()

In [ ]:
result.dtypes

In [ ]:
result["평점"]=pd.to_numeric(result["평점"]) 
result["평점수"]=pd.to_numeric(result["평점수"]) 

result.dtypes

In [ ]:
#중복제거
result=result.drop_duplicates(['평점수'],keep = 'first')
result

In [ ]:
# 저장하기
result.to_excel("iherb_mctoil.xlsx", encoding='utf-8-sig')
# df.to_csv("blog_url.csv")

### 1.1.2 평균필터링 

In [ ]:
mean_df = result.mean()
print(mean_df)

In [ ]:
condition1 = result.평점  >= 4.51
condition2 = result.평점수 >= 602
df = result.loc[condition1, ['item_id','평점','평점수','리뷰url']]
df = result.loc[condition2, ['item_id','평점','평점수','리뷰url']]
print(df)
print("")
print("데이터길이: ",len(df))

In [ ]:
# 저장하기
df.to_excel("iherb_mctoil-filtered.xlsx", encoding='utf-8-sig')

## 1.2 유저리뷰 크롤링

In [ ]:
#데이터로드 
df_load = pd.read_excel("iherb_mctoil-filtered.xlsx")
df_load = df_load.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제

df_load

In [ ]:
%%time
# 수집할 글 갯수 정하기
user_list=[]
rev_count_list=[]
driver = webdriver.Chrome(chrome_path)

# 수집한 url 돌면서 데이터 수집
for k in tqdm_notebook(range(0,4)):
        
    # 리뷰창 띄우기
    url=df_load.리뷰url[k]
    driver.get(url)   
    time.sleep(2)
    
    for j in range(0,20):

        for i in range(0, 10):
            # 유저아이디링크따기
            userid =".customer-profile-link"
            user=driver.find_elements_by_css_selector(userid)
            userurl=user[i].get_attribute('href')
            user_list.append(userurl)

            #유저리뷰수 따기
            rev_count=driver.find_elements_by_css_selector(".snapshot-count")
            count= rev_count[i].text
            rev_count_list.append(count)

            if i == 9:
                pg_nx= driver.find_element_by_css_selector(".paging-btn-next")
                driver.execute_script("arguments[0].click();", pg_nx)
                time.sleep(2)
    print(df_load.item_id[k], len(user_list))

In [ ]:
df1= pd.DataFrame({"고객링크":user_list,"리뷰수":rev_count_list})
df1

In [ ]:
%%time
# 수집할 글 갯수 정하기
user_list=[]
rev_count_list=[]
driver = webdriver.Chrome(chrome_path)

# 수집한 url 돌면서 데이터 수집
for k in tqdm_notebook(range(4,8)):
        
    # 리뷰창 띄우기
    url=df_load.리뷰url[k]
    driver.get(url)   
    time.sleep(2)

    
    for j in range(0,20):


        for i in range(0, 10):
            # 유저아이디링크따기
            userid =".customer-profile-link"
            user=driver.find_elements_by_css_selector(userid)
            userurl=user[i].get_attribute('href')
            user_list.append(userurl)

            #유저리뷰수 따기
            rev_count=driver.find_elements_by_css_selector(".snapshot-count")
            count= rev_count[i].text
            rev_count_list.append(count)
            
            if i == 9:
                pg_nx= driver.find_element_by_css_selector(".paging-btn-next")
                driver.execute_script("arguments[0].click();", pg_nx)
                time.sleep(2)
    print(df_load.item_id[k], len(user_list))

In [ ]:
df2= pd.DataFrame({"고객링크":user_list,"리뷰수":rev_count_list})
df2

In [ ]:
df_user=pd.concat([df1,df2],axis=0)
df_user

In [ ]:
#중복제거
df_user=df_user.drop_duplicates(['고객링크'],keep = 'first')
df_user

### 1.2.1 유저리뷰 크롤링 전처리

In [174]:
#숫자화
df_user["리뷰수"]=pd.to_numeric(df_user["리뷰수"]) 

In [175]:
#리뷰수 10건이상필터링
df_user_10 = df_user.loc[(df_user.리뷰수>=10), ['고객링크','리뷰수']]
df_user_10

,고객링크,리뷰수
0,https://kr.iherb.com/me/aminochan,275
3,https://kr.iherb.com/me/4680214676756300470,512
8,https://kr.iherb.com/me/A22434FB52,16
10,https://kr.iherb.com/me/4883425644413566162,84
11,https://kr.iherb.com/me/5200700960892458955,32
...,...,...
780,https://kr.iherb.com/me/D846920492,93
783,https://kr.iherb.com/me/5614149106978046441,36
788,https://kr.iherb.com/me/4790040468155740968,44
790,https://kr.iherb.com/me/0479329787,23


In [176]:
df_user.to_excel("iherb_userlist.xlsx", encoding='utf-8-sig')
df_user_10.to_excel("iherb_userlist_morethan10.xlsx", encoding='utf-8-sig')

## 1.3 유저 아이템 크롤링

In [177]:
#데이터로드 
df_load1 = pd.read_excel("iherb_userlist_morethan10.xlsx")
df_load1 = df_load1.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제

df_load1

,고객링크,리뷰수
0,https://kr.iherb.com/me/aminochan,275
1,https://kr.iherb.com/me/4680214676756300470,512
2,https://kr.iherb.com/me/A22434FB52,16
3,https://kr.iherb.com/me/4883425644413566162,84
4,https://kr.iherb.com/me/5200700960892458955,32
...,...,...
570,https://kr.iherb.com/me/D846920492,93
571,https://kr.iherb.com/me/5614149106978046441,36
572,https://kr.iherb.com/me/4790040468155740968,44
573,https://kr.iherb.com/me/0479329787,23


In [178]:
len(df_load1)

575

In [179]:
%%time
# 수집할 글 갯수 정하기
rev_item_list=[]
rev_item_rating_list=[]
userid_list=[]
driver = webdriver.Chrome(chrome_path)

# 수집한 url 돌면서 데이터 수집
for k in tqdm_notebook(range(0,len(df_load))):
    
    
    # 리뷰창 띄우기
    url=df_load1.고객링크[k]
    driver.get(url)   
    time.sleep(2)
    
    try:
        for j in range(0,100):
            for i in range(0, 10):
                # 글 띄우기

                rev_item=driver.find_elements_by_css_selector(".product-review-name")
                element1 =rev_item[i].text
                rev_item_list.append(element1)

                rev_item_rating=driver.find_elements_by_css_selector("div.col-xs-14.review-content > div > div.rating > a>svg>use")
                element2 =rev_item_rating[i].get_attribute("xlink:href")
                rev_item_rating_list.append(element2)

                userid_list.append(k)
                print(element2)
            
                if i == 9:
                    pg_nx= driver.find_element_by_css_selector(".pagination-right")
                    driver.execute_script("arguments[0].click();", pg_nx)
                    time.sleep(2)
                    
    except:
        print(k)

  0%|          | 0/575 [00:00<?, ?it/s]

#icon-stars_50
#icon-stars_50
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_30
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_30
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_30
#icon-stars_50
#icon-stars_30
#icon-stars_40
#icon-stars_20
#icon-stars_50
#icon-stars_30
#icon-star

#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
9
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_30
#icon-stars_30
#icon-stars_50
#icon-stars_50
10
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon

#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_40
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_20
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_40
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_40
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-star

#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-star

...

#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_30
#icon-stars_50
#icon-stars_30
#icon-stars_30
#icon-stars_40
#icon-stars_30
#icon-stars_40
#icon-stars_40
#icon-stars_30
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_30
#icon-stars_50
#icon-stars_30
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_30
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-star

#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-star

#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
415
416
417
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_50
#icon-stars_40
#icon-stars_30
#icon-stars_40
#icon-stars_40
#icon-stars_30
#icon-stars_40
#icon-stars_40
#icon-stars_30
#icon-stars_50
#icon-stars_40
#icon-stars_30
#icon-stars_30
#icon-stars_50
#icon-stars_30
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_20
#icon-stars_20
#icon-stars_30
#icon-stars_50
#icon-stars_30
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_4

#icon-stars_50
423


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=98.0.4758.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x0121A0A3+2597027]
	Ordinal0 [0x011ABA61+2144865]
	Ordinal0 [0x010A3C58+1064024]
	Ordinal0 [0x01095992+1005970]
	Ordinal0 [0x0109502B+1003563]
	Ordinal0 [0x010A960D+1086989]
	Ordinal0 [0x010F8DE6+1412582]
	Ordinal0 [0x010E85C6+1344966]
	Ordinal0 [0x010C54C6+1201350]
	Ordinal0 [0x010C63B6+1205174]
	GetHandleVerifier [0x013C10A2+1681842]
	GetHandleVerifier [0x01473B0C+2413596]
	GetHandleVerifier [0x012B1221+568113]
	GetHandleVerifier [0x012AFFA3+563379]
	Ordinal0 [0x011B174E+2168654]
	Ordinal0 [0x011B6538+2188600]
	Ordinal0 [0x011B6680+2188928]
	Ordinal0 [0x011C030C+2229004]
	BaseThreadInitThunk [0x771DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77627A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77627A6E+238]


In [196]:
ratings=pd.DataFrame({"userid":userid_list,"item":rev_item_list, "rating":rev_item_rating_list})
ratings.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36649 entries, 0 to 36648
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   userid  36649 non-null  int64 
 1   item    36649 non-null  object
 2   rating  36649 non-null  object
dtypes: int64(1), object(2)
memory usage: 859.1+ KB


In [197]:
rating= ratings["rating"].str.split('_').str[1].tolist()
ratings["rating"]=rating
ratings

,userid,item,rating
0,0,"Sierra Fit,프로틴 크리스프, 솔티드 토피, 12개입, 각 56g(1.98oz)",50
1,0,"Sierra Fit,프로틴 크리스프, 땅콩버터, 바 12개입, 각 56g(1.98oz)",50
2,0,"Sierra Fit,프로틴 크리스프, 초콜릿 칩 쿠키 도우, 12개입, 각 56g(...",30
3,0,"Sierra Fit,유청 단백질 컴플리트, 바닐라, 2.27kg(5lb)",50
4,0,"Bob's Red Mill,Organic Quick Cooking Rolled Oa...",50
...,...,...,...
36644,423,"Country Life,글라이신, 500mg, 100정",50
36645,423,"Sports Research,MCT 오일, 무맛, 32 액량 온스 (946 ml)",50
36646,423,"NOW Foods,Real Food, 무가공 아몬드 가루, 624g(22oz)",50
36647,423,"Garden of Life,풀로 키운 콜라겐 펩타이드, 향 무첨가, 19.75 oz...",50


In [198]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36649 entries, 0 to 36648
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   userid  36649 non-null  int64 
 1   item    36649 non-null  object
 2   rating  36649 non-null  object
dtypes: int64(1), object(2)
memory usage: 859.1+ KB


In [200]:
ratings["rating"]=pd.to_numeric(ratings["rating"])
ratings["rating"]=ratings["rating"]/10
ratings

,userid,item,rating
0,0,"Sierra Fit,프로틴 크리스프, 솔티드 토피, 12개입, 각 56g(1.98oz)",5.0
1,0,"Sierra Fit,프로틴 크리스프, 땅콩버터, 바 12개입, 각 56g(1.98oz)",5.0
2,0,"Sierra Fit,프로틴 크리스프, 초콜릿 칩 쿠키 도우, 12개입, 각 56g(...",3.0
3,0,"Sierra Fit,유청 단백질 컴플리트, 바닐라, 2.27kg(5lb)",5.0
4,0,"Bob's Red Mill,Organic Quick Cooking Rolled Oa...",5.0
...,...,...,...
36644,423,"Country Life,글라이신, 500mg, 100정",5.0
36645,423,"Sports Research,MCT 오일, 무맛, 32 액량 온스 (946 ml)",5.0
36646,423,"NOW Foods,Real Food, 무가공 아몬드 가루, 624g(22oz)",5.0
36647,423,"Garden of Life,풀로 키운 콜라겐 펩타이드, 향 무첨가, 19.75 oz...",5.0


In [201]:
#저장하기
ratings.to_excel("iherb_ratings.xlsx", encoding='utf-8-sig')

# 2. CF추천알고리즘 전처리

## 2.1 사용자-아이템 평점행렬전환

In [202]:
#행렬변환
ratings_matrix = ratings.pivot_table('rating', index='userid', columns='item')

print(ratings_matrix.shape)
ratings_matrix

(392, 12528)


item,"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)","1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)","1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)","1Kvit-C,비타민C, 에센셜, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.90g(0.24oz)","1Kvit-C,비타민C, 필수, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 3팩, 각 6.90g(0.24oz)","21st Century,600+D3 플러스 미네랄, 120정","21st Century,600+D3, 칼슘 & 비타민D3 보충제, 75정","21st Century,600+D3, 칼슘 및 비타민D3 보충제, 400정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 코팅 정제 120정","21st Century,B-100 복합체, 서방정, 타블렛 60정",...,"iHerb Goods,홍보용 메모 액세서리, 3 피스","iHerb Nutritionals,비타민 D3, 5,000 IU, 120 소프트젤","iUNIK,Propolis Vitamin Sleeping Mask, 60 ml","pHion Balance,애씨드 드레이니지, 캡슐 60정","petnc NATURAL CARE,알래스카 야생 연어 오일, 강아지용, 532ml(18oz)","petnc NATURAL CARE,엉덩이 및 관절, 모든 견종, 간, 소프트 츄 90정","petnc NATURAL CARE,클린 이어 클렌징 패드, 고양이 및 강아지용, 패드 90개","petnc NATURAL CARE,펫 내츄럴 케어, 시니어 종합 비타민 포뮬라, 시니어 도그, 간(Liver) 맛, 60 츄어블 정","theBalm Cosmetics,바하마 마마, 브론저, 섀도 & 컨투어 파우더, 0.25oz(7.08g)","theBalm Cosmetics,핫 마마, 섀도우/블러시, 0.25oz(7.08g)"
userid,,,,,,,,,,,,,,,,,,,,,
0,NaN,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
#nan값변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

item,"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)","1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)","1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)","1Kvit-C,비타민C, 에센셜, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.90g(0.24oz)","1Kvit-C,비타민C, 필수, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 3팩, 각 6.90g(0.24oz)","21st Century,600+D3 플러스 미네랄, 120정","21st Century,600+D3, 칼슘 & 비타민D3 보충제, 75정","21st Century,600+D3, 칼슘 및 비타민D3 보충제, 400정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 코팅 정제 120정","21st Century,B-100 복합체, 서방정, 타블렛 60정",...,"iHerb Goods,홍보용 메모 액세서리, 3 피스","iHerb Nutritionals,비타민 D3, 5,000 IU, 120 소프트젤","iUNIK,Propolis Vitamin Sleeping Mask, 60 ml","pHion Balance,애씨드 드레이니지, 캡슐 60정","petnc NATURAL CARE,알래스카 야생 연어 오일, 강아지용, 532ml(18oz)","petnc NATURAL CARE,엉덩이 및 관절, 모든 견종, 간, 소프트 츄 90정","petnc NATURAL CARE,클린 이어 클렌징 패드, 고양이 및 강아지용, 패드 90개","petnc NATURAL CARE,펫 내츄럴 케어, 시니어 종합 비타민 포뮬라, 시니어 도그, 간(Liver) 맛, 60 츄어블 정","theBalm Cosmetics,바하마 마마, 브론저, 섀도 & 컨투어 파우더, 0.25oz(7.08g)","theBalm Cosmetics,핫 마마, 섀도우/블러시, 0.25oz(7.08g)"
userid,,,,,,,,,,,,,,,,,,,,,
0,0.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2.2 아이템간 유사도산출

In [204]:
# 아이템-사용자 행렬로 transpose 한다.
ratings_matrix_T = ratings_matrix.transpose()  # 전치 행렬

print(ratings_matrix_T.shape)
ratings_matrix_T.head(3)

(12528, 392)


userid,0,1,2,3,4,5,6,8,9,10,...,412,413,414,415,418,419,420,421,422,423
item,,,,,,,,,,,,,,,,,,,,,
"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)",4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)",4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [205]:
# 영화와 영화들 간 코사인 유사도 산출
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(12528, 12528)


item,"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)","1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)","1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)","1Kvit-C,비타민C, 에센셜, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.90g(0.24oz)","1Kvit-C,비타민C, 필수, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 3팩, 각 6.90g(0.24oz)","21st Century,600+D3 플러스 미네랄, 120정","21st Century,600+D3, 칼슘 & 비타민D3 보충제, 75정","21st Century,600+D3, 칼슘 및 비타민D3 보충제, 400정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 코팅 정제 120정","21st Century,B-100 복합체, 서방정, 타블렛 60정",...,"iHerb Goods,홍보용 메모 액세서리, 3 피스","iHerb Nutritionals,비타민 D3, 5,000 IU, 120 소프트젤","iUNIK,Propolis Vitamin Sleeping Mask, 60 ml","pHion Balance,애씨드 드레이니지, 캡슐 60정","petnc NATURAL CARE,알래스카 야생 연어 오일, 강아지용, 532ml(18oz)","petnc NATURAL CARE,엉덩이 및 관절, 모든 견종, 간, 소프트 츄 90정","petnc NATURAL CARE,클린 이어 클렌징 패드, 고양이 및 강아지용, 패드 90개","petnc NATURAL CARE,펫 내츄럴 케어, 시니어 종합 비타민 포뮬라, 시니어 도그, 간(Liver) 맛, 60 츄어블 정","theBalm Cosmetics,바하마 마마, 브론저, 섀도 & 컨투어 파우더, 0.25oz(7.08g)","theBalm Cosmetics,핫 마마, 섀도우/블러시, 0.25oz(7.08g)"
item,,,,,,,,,,,,,,,,,,,,,
"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)",1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.135317,0.000000,0.0,0.624695,0.0,0.0,0.0,0.624695,0.0,0.0
"1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)",0.0,1.0,1.0,0.441726,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.408248,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
"1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)",0.0,1.0,1.0,0.441726,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.408248,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


# 3. 아이템 기반 CF-KNN

In [206]:
# 평점 벡터(행 벡터)와 유사도 벡터(열 벡터)를 내적(dot)해서 예측 평점을 계산하는 함수 정의
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [207]:
item_sim_df.head(3)

item,"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)","1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)","1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)","1Kvit-C,비타민C, 에센셜, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.90g(0.24oz)","1Kvit-C,비타민C, 필수, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 3팩, 각 6.90g(0.24oz)","21st Century,600+D3 플러스 미네랄, 120정","21st Century,600+D3, 칼슘 & 비타민D3 보충제, 75정","21st Century,600+D3, 칼슘 및 비타민D3 보충제, 400정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 코팅 정제 120정","21st Century,B-100 복합체, 서방정, 타블렛 60정",...,"iHerb Goods,홍보용 메모 액세서리, 3 피스","iHerb Nutritionals,비타민 D3, 5,000 IU, 120 소프트젤","iUNIK,Propolis Vitamin Sleeping Mask, 60 ml","pHion Balance,애씨드 드레이니지, 캡슐 60정","petnc NATURAL CARE,알래스카 야생 연어 오일, 강아지용, 532ml(18oz)","petnc NATURAL CARE,엉덩이 및 관절, 모든 견종, 간, 소프트 츄 90정","petnc NATURAL CARE,클린 이어 클렌징 패드, 고양이 및 강아지용, 패드 90개","petnc NATURAL CARE,펫 내츄럴 케어, 시니어 종합 비타민 포뮬라, 시니어 도그, 간(Liver) 맛, 60 츄어블 정","theBalm Cosmetics,바하마 마마, 브론저, 섀도 & 컨투어 파우더, 0.25oz(7.08g)","theBalm Cosmetics,핫 마마, 섀도우/블러시, 0.25oz(7.08g)"
item,,,,,,,,,,,,,,,,,,,,,
"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)",1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.135317,0.000000,0.0,0.624695,0.0,0.0,0.0,0.624695,0.0,0.0
"1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)",0.0,1.0,1.0,0.441726,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.408248,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
"1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)",0.0,1.0,1.0,0.441726,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.408248,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [208]:
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred

array([[0.14012611, 3.4829422 , 3.4829422 , ..., 0.17213945, 0.10811539,
        0.10811539],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00534547,
        0.00534547],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02057956, 0.02249167, 0.02249167, ..., 0.        , 0.03309283,
        0.03309283],
       [0.1027691 , 0.04094913, 0.04094913, ..., 0.18955829, 0.19699836,
        0.19699836],
       [0.        , 0.        , 0.        , ..., 0.        , 0.01049323,
        0.01049323]])

In [209]:
# 데이터프레임으로 변환
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
print(ratings_pred_matrix.shape)
ratings_pred_matrix.head(10)

(392, 12528)


item,"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)","1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)","1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)","1Kvit-C,비타민C, 에센셜, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.90g(0.24oz)","1Kvit-C,비타민C, 필수, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 3팩, 각 6.90g(0.24oz)","21st Century,600+D3 플러스 미네랄, 120정","21st Century,600+D3, 칼슘 & 비타민D3 보충제, 75정","21st Century,600+D3, 칼슘 및 비타민D3 보충제, 400정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 코팅 정제 120정","21st Century,B-100 복합체, 서방정, 타블렛 60정",...,"iHerb Goods,홍보용 메모 액세서리, 3 피스","iHerb Nutritionals,비타민 D3, 5,000 IU, 120 소프트젤","iUNIK,Propolis Vitamin Sleeping Mask, 60 ml","pHion Balance,애씨드 드레이니지, 캡슐 60정","petnc NATURAL CARE,알래스카 야생 연어 오일, 강아지용, 532ml(18oz)","petnc NATURAL CARE,엉덩이 및 관절, 모든 견종, 간, 소프트 츄 90정","petnc NATURAL CARE,클린 이어 클렌징 패드, 고양이 및 강아지용, 패드 90개","petnc NATURAL CARE,펫 내츄럴 케어, 시니어 종합 비타민 포뮬라, 시니어 도그, 간(Liver) 맛, 60 츄어블 정","theBalm Cosmetics,바하마 마마, 브론저, 섀도 & 컨투어 파우더, 0.25oz(7.08g)","theBalm Cosmetics,핫 마마, 섀도우/블러시, 0.25oz(7.08g)"
userid,,,,,,,,,,,,,,,,,,,,,
0,0.140126,3.482942,3.482942,2.503448,0.048749,0.069934,0.072439,0.102465,0.054535,0.078241,...,0.070659,2.345509,0.000000,0.172139,0.199213,0.050227,0.199213,0.172139,0.108115,0.108115
1,0.000000,0.000000,0.000000,0.007458,0.003154,0.000000,0.000000,0.001588,0.002610,0.003050,...,0.003396,0.001615,0.000000,0.000000,0.004679,0.005912,0.004679,0.000000,0.005345,0.005345
2,0.000000,0.000000,0.000000,0.000000,0.001752,0.000000,0.000000,0.000000,0.005185,0.003163,...,0.000980,0.002018,0.000000,0.000000,0.005848,0.004777,0.005848,0.000000,0.000000,0.000000
3,0.034914,0.023170,0.023170,0.044059,0.046315,0.022574,0.027806,0.038672,0.042571,0.056042,...,0.039937,0.016644,0.000000,0.045794,0.043205,0.056725,0.043205,0.045794,0.108079,0.108079
4,0.120623,0.027061,0.027061,0.022761,0.045150,0.020810,0.021480,0.034571,0.042025,0.048967,...,0.037189,0.012839,0.000000,0.169230,0.026680,0.029974,0.026680,0.169230,0.035029,0.035029
5,0.039321,0.007068,0.007068,0.006376,0.003411,0.001386,0.001639,0.000000,0.002436,0.000734,...,0.003959,0.002335,0.000000,0.042931,0.006994,0.006940,0.006994,0.042931,0.000000,0.000000
6,0.019951,0.005654,0.005654,0.010236,0.007894,0.004156,0.005703,0.000000,0.005686,0.001943,...,0.004108,0.005773,0.000000,0.034344,0.000000,0.005616,0.000000,0.034344,0.031065,0.031065
8,0.043831,0.028536,0.028536,0.055485,0.027995,0.016995,0.019733,0.015022,0.028100,0.025289,...,0.022270,0.016299,0.005049,0.108886,0.040324,0.027006,0.040324,0.108886,0.045922,0.045922
9,0.130388,0.055225,0.055225,0.066665,0.037672,0.017892,0.021254,0.033723,0.046366,0.054566,...,0.037655,0.036561,0.000000,0.124626,0.034485,0.053292,0.034485,0.124626,0.055655,0.055655


## 3.1 예측 평점 정확도를 판단하기 위해 오차 함수인 RMSE를 이용

In [210]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))


아이템 기반 모든 인접 이웃 MSE:  12.183634659683776


## 3.2 top-n 유사도를 가진 데이터들에 대해서만 예측 평점 계산

In [239]:
# 사용자 9번에게 영화를 추천해보자
# 추천에 앞서 9번 사용자가 높은 평점을 준 영화를 확인해보면
user_rating_id = ratings_matrix.loc[9, :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

item
Atkins,Endulge, Chocolate Covered Almonds, 5 Packs, 1 oz (28 g) Each    5.0
NOW Foods,칼슘 및 마그네슘, 250정                                               5.0
Lake Avenue Nutrition,CoQ10, Bioperine 함유, 100mg, 베지 소프트젤 365정          5.0
Lake Avenue Nutrition,밀크시슬 추출물, 175mg, 베지 캡슐 240정                       5.0
Lake Avenue Nutrition,비타민 K2(메나퀴논-7), 50mcg, 식물성 소프트젤 360정              5.0
Lake Avenue Nutrition,아스타잔틴, 10mg, 베지 소프트젤 120정                         5.0
NOW Foods,EGCg, 녹차추출물, 400mg, 베지 캡슐 180정                                5.0
NOW Foods,Real Food, 무가공 아몬드 가루, 624g(22oz)                             5.0
NOW Foods,비오틴, 5,000mcg, 베지 캡슐 120정                                     5.0
NOW Foods,비타민A, 25,000IU, 소프트젤 250정                                     5.0
Name: 9, dtype: float64

In [240]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred


In [241]:
# 실행시간 2분 정도 걸림
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

아이템 기반 인접 TOP-20 이웃 MSE:  7.458609238645662


## 3.3 사용자 상품 추천

### 사용자가 미사용한 상품 추천

In [242]:
# 사용자가 미사용한 상품 추천user_rating이 0보다 크면 기존에 사용한상품 
def get_unused_item(ratings_matrix, userid):
    
    user_rating = ratings_matrix.loc[userid,:]
    already_used = user_rating[ user_rating > 0].index.tolist()
    
    items_list = ratings_matrix.columns.tolist()
    unused_list = [ item for item in items_list if item not in already_used]
    
    return unused_list


In [243]:
def recomm_item_by_userid(pred_df, userid, unused_list, top_n=10):

    recomm_items = pred_df.loc[userid, unused_list].sort_values(ascending=False)[:top_n]
    return recomm_items

In [245]:
unused_list = get_unused_item(ratings_matrix, 9)
recomm_items = recomm_item_by_userid(ratings_pred_matrix, 9, unused_list, top_n=10)
recomm_items = pd.DataFrame(data=recomm_items.values,index=recomm_items.index, columns=['pred_score'])
recomm_items

,pred_score
item,
"California Gold Nutrition,LactoBif 프로바이오틱스, 50억 CFU, 베지 캡슐 10정",2.010667
"iHerb Goods,스테인리스 스틸 티 인퓨저",1.940888
"California Gold Nutrition,비타민D3, 125mcg(5,000IU), 피쉬 젤라틴 소프트젤 90정",1.529171
"California Gold Nutrition,CollagenUP, 저분자 해양 콜라겐 + 히알루론산 + 비타민C, 무맛, 206g(7.26oz)",1.477201
"NOW Foods,카프릴산, 600mg, 소프트젤 100정",1.449773
"NOW Foods,Sports, MCT 오일, 무맛, 473ml(16fl oz)",1.403558
"California Gold Nutrition,CollagenUP, 해양 저분자 콜라겐 + 히알루론산 + 비타민C, 무맛, 464g (16.37oz)",1.389570
"California Gold Nutrition,LactoBif 프로바이오틱스, 300억 CFU, 베지 캡슐 60정",1.378339
"NOW Foods,Sports, 중쇄중성지방 오일, 946ml(32fl oz)",1.246222


# 4. surprise이용 추천시스템

## 2. 실제 영화 평점 데이터로 surprise 패키지 사용해보기

In [250]:
from surprise import SVD      # 행렬 분해 알고리즘 
from surprise import Dataset  # 내장 데이터 불러오는 라이브러리
from surprise import accuracy # rmse로 예측 에러 평가
from surprise.model_selection import train_test_split  # 학습셋, 테스트셋 분리

### 4.1 Cross Validation(교차 검증)과 GridSearchCV(하이퍼 파라미터 튜닝)

#### Cross Validation(교차 검증)

In [275]:
from surprise import SVD      # MF(행렬 분해) 알고리즘 
from surprise import Dataset  # 내장 데이터 불러오는 라이브러리
from surprise import accuracy # accuracy의 rmse메소드로 예측 에러 평가
from surprise.model_selection import train_test_split  # 학습셋, 테스트셋 분리
from surprise.model_selection import cross_validate

In [276]:
%%time
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_excel("iherb_ratings.xlsx")
ratings = ratings.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제

print(ratings.shape)
ratings.head()

(36649, 3)
Wall time: 3.56 s


,userid,item,rating
0,0,"Sierra Fit,프로틴 크리스프, 솔티드 토피, 12개입, 각 56g(1.98oz)",5
1,0,"Sierra Fit,프로틴 크리스프, 땅콩버터, 바 12개입, 각 56g(1.98oz)",5
2,0,"Sierra Fit,프로틴 크리스프, 초콜릿 칩 쿠키 도우, 12개입, 각 56g(...",3
3,0,"Sierra Fit,유청 단백질 컴플리트, 바닐라, 2.27kg(5lb)",5
4,0,"Bob's Red Mill,Organic Quick Cooking Rolled Oa...",5


In [277]:
# 데이터 로드하기
reader = Reader(rating_scale=(0.5, 5.0))
# ratings DataFrame 에서 컬럼은 userId, itemId, rating 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userid', 'item', 'rating']], reader)

# 데이터를 학습셋, 테스트셋으로 나누기
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 행렬 분해 알고리즘 학습 및 예측, 평가(RMSE)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

RMSE: 0.5445
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5673  0.5620  0.5690  0.5331  0.5761  0.5615  0.0149  
MAE (testset)     0.3276  0.3283  0.3263  0.3149  0.3297  0.3254  0.0053  
Fit time          1.50    1.85    1.70    1.59    1.49    1.63    0.14    
Test time         0.06    0.08    0.06    0.06    0.06    0.06    0.01    


{'test_rmse': array([0.56728035, 0.56195026, 0.56897878, 0.53307086, 0.57606513]),
 'test_mae': array([0.32762717, 0.32827168, 0.32631811, 0.31490271, 0.32969704]),
 'fit_time': (1.5009839534759521,
  1.8510513305664062,
  1.7044384479522705,
  1.5897481441497803,
  1.494006872177124),
 'test_time': (0.06386613845825195,
  0.07579421997070312,
  0.0608372688293457,
  0.06482601165771484,
  0.05884146690368652)}

#### GridSearchCV(하이퍼 파라미터 튜닝)

In [278]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# GridSearchCV 세팅 : CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs

In [279]:
%%time
# GridSearchCV로 학습
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

# 20만 건 데이터 기준 실행시간 (원본 데이터 2천만 건이면 8분~10분 정도 걸림)

0.5623133480159631
{'n_epochs': 20, 'n_factors': 50}
Wall time: 2min 12s


## 4. Surprise 를 이용한 개인화 영화 추천 시스템 구축

### 전체 데이터로 학습 진행(학습셋, 테스트셋 나누지 않고)

In [280]:
%%time
from surprise.dataset import DatasetAutoFolds  # 데이터 세트 전체를 학습 데이터로 사용할 수 있게 해주는 라이브러리

reader = Reader(line_format='user item rating', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds.load_from_df(ratings[['userid', 'item', 'rating']], reader)

# 전체 데이터를 학습데이터로 생성함.
trainset = data_folds.build_full_trainset()
# 실행시간 3분

Wall time: 138 ms


In [281]:
%%time
# SVD 협업필터링으로 추천모델 학습(하이퍼 파라미터는 앞서 그리드서치로 구한 것들)
algo = SVD(n_epochs=20, n_factors=50, random_state=0)

algo.fit(trainset)
# 실행시간 11분

Wall time: 1.9 s


In [282]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
items = pd.read_excel('iherb_ratings.xlsx')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
itemIds = ratings[ratings['userId']==9]['item']
if itemIds.count() == 0:
    print('사용자 아이디 9는==4 영화 아이디 42의 평점 없음')

print(item[item['item']==42])

KeyError: 'userId'

In [79]:
# predict 메소드를 사용해서 예측 평점 구하기
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 2.95   {'was_impossible': False}


### 특정 사용자가 아직 안 본 전체 영화 추출 후 예측 평점 순으로 영화 추천

In [80]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 1 추천대상 영화수: 27277 전체 영화수: 27278


In [81]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화(27243개)에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [82]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print("")
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 1 추천대상 영화수: 27277 전체 영화수: 27278

##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.403255960567584
Braveheart (1995) : 4.354730709188801
Pulp Fiction (1994) : 4.279467874657644
Shawshank Redemption, The (1994) : 4.2719954041396715
Schindler's List (1993) : 4.268033622651485
Silence of the Lambs, The (1991) : 4.218225009929055
Godfather, The (1972) : 4.216492633386306
Lord of the Rings: The Return of the King, The (2003) : 4.206242413033943
Personal Journey with Martin Scorsese Through American Movies, A (1995) : 4.204346268855151
Frozen Planet (2011) : 4.197629744293488
